<a href="https://colab.research.google.com/github/srirambandi/ai/blob/master/examples/cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# when running in colab notebooks, first install library
!pip install import-ai
# upload respective dataset manually from examples directory of the library or download as below
!apt install subversion
!svn checkout https://github.com/srirambandi/ai/trunk/examples/CIFAR10

In [0]:
import ai
import numpy as np

In [0]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [0]:
train_file = 'CIFAR10/data_batch_'
test_file = 'CIFAR10/test_batch'

In [0]:
ai.manual_seed(2357)

In [0]:
class CIFAR10(ai.Module):
    def __init__(self):
        self.conv1 = ai.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = ai.Conv2d(32, 32, kernel_size=3, stride=1)
        self.conv3 = ai.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv4 = ai.Conv2d(64, 64, kernel_size=3)
        self.fc1 = ai.Linear(2304, 512)
        self.fc2 = ai.Linear(512, 10)
        
    def forward(self, x):
        o1 = ai.G.relu(self.conv1.forward(x))
        o2 = ai.G.relu(self.conv2.forward(o1))
        o3 = ai.G.dropout(ai.G.maxpool2d(o2), p=0.75)
        o4 = ai.G.relu(self.conv3.forward(o3))
        o5 = ai.G.relu(self.conv4.forward(o4))
        o6 = ai.G.dropout(ai.G.maxpool2d(o5), p=0.75)
        o7 = ai.G.dropout(ai.G.relu(self.fc1.forward(o6)), p=0.5)
        o8 = ai.G.softmax(self.fc2.forward(o7))
        return o8

In [20]:
cifar10 = CIFAR10()
print(cifar10)

CIFAR10(
  conv1: Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=True)
  conv2: Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(0, 0), bias=True)
  conv3: Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=True)
  conv4: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(0, 0), bias=True)
  fc1: Linear(input_features=2304, output_features=512, bias=True)
  fc2: Linear(input_features=512, output_features=10, bias=True)
)


In [0]:
L = ai.Loss(loss_fn='CrossEntropyLoss')
optim = ai.Optimizer(cifar10.parameters(), optim_fn='Adam', lr=1e-3)

In [0]:
it, epoch = 0, 0
m = 128

In [0]:
def evaluate():
    ai.G.grad_mode = False
    file = test_file
    dict = unpickle(file)
    inputs = dict[b'data']
    outputs = dict[b'labels']
    correct, total = 0, 0
    test_m = m
    for batch in range(int(len(outputs) / m)):
        input = inputs[batch * test_m : (batch + 1) * test_m].reshape(test_m, 3, 32, 32) / 255
        input =  np.stack([_ for _ in input], axis = -1)
        output = np.array(outputs[batch * test_m : (batch + 1) * test_m])
        scores = cifar10.forward(input)
        preds = np.argmax(scores.data, axis=0)
        correct += np.sum(np.equal(output, preds))
        total += test_m
    accuracy = float(correct / total)
    ai.G.grad_mode = True
    return accuracy

In [0]:
while epoch < 5:
    epoch += 1
    it = 0
    for set in range(1, 6):
        print('Set #{} started.'.format(set))
        dataset = train_file + str(set)
        dict = unpickle(dataset)
        inputs = dict[b'data']
        outputs = dict[b'labels']
        for batch in range(int(len(outputs) / m)):
            input = inputs[batch * m : (batch + 1) * m].reshape(m, 3, 32, 32) / 255
            input =  np.stack([_ for _ in input], axis = -1)
            output = outputs[batch * m : (batch + 1) * m]
            onehot = np.zeros((10, m))
            for _ in range(m):
                onehot[output[_], _] = 1.0
            scores = cifar10.forward(input)
            loss = L.loss(scores, onehot)
            loss.backward()
            optim.step()        # update parameters with optimization functions
            optim.zero_grad()   # clearing the backprop list and resetting the gradients to zero
            if it%25 == 0:
                print('epoch: {}, iter: {}, loss: {}'.format(epoch, it, loss.data[0][0]))
            it += 1
    print('Epoch {} completed. Accuracy {:.2%} \n'.format(epoch, evaluate()))

In [0]:
cifar10.save()